## Pytorch Basics

载入基本模块

In [1]:
import torch
import numpy as np

### torch 基本处理单元

**Tensor** 类是 torch 中的基本数据类型。在 0.4.0 版本开始，**Variable** 类和 **Tensor** 类开始合并统一，且自 0.4.0 版本开始，允许生成 0 维 tensor 变量。

我们可以调用该类的实例化函数，得到一个 5x4 的矩阵

In [2]:
torch.Tensor(5, 4)

tensor([[ 3.4382e-37,  0.0000e+00,  5.7453e-44,  0.0000e+00],
        [        nan,  6.4460e-44,  1.3733e-14,  6.4076e+07],
        [ 2.0706e-19,  7.3909e+22,  2.4176e-12,  1.1625e+33],
        [ 8.9605e-01,  1.1632e+33,  5.6003e-02,  7.0374e+22],
        [ 5.7453e-44,  5.7453e-44,  4.4721e+21,  2.8776e+32]])

### 变量的创建

目前，tensor 对象的创建支持多种方式，例如 `torch.tensor()` 可以直接从 _list, tuple, np.ndarray, scalar_ 等生成 tensor 变量。而 `torch.from_numpy()` 则可以从 np.ndarray 获得一个数据的**浅拷贝**。其余也有类似 `torch.ones()`, `torch.zeros()`, `torch.arange()`, `torch.linspace()`, `torch.eye()` 等数据生成方式。同时也包括一些随机生成方式，可以参照文档。下面的例子就是返回一个 5x4 的随机矩阵，随机初始化为 0-1 的均匀分布

In [3]:
a = torch.rand(5, 4)
print(a)

tensor([[ 0.5879,  0.4390,  0.4041,  0.4001],
        [ 0.4700,  0.1505,  0.0929,  0.3115],
        [ 0.4930,  0.9751,  0.7643,  0.9618],
        [ 0.2723,  0.1353,  0.4302,  0.3362],
        [ 0.8675,  0.0024,  0.0486,  0.8003]])


获取一个矩阵的 size

In [4]:
a.size()

torch.Size([5, 4])

#### 注意

比较 tensor 的类型时不能再使用 `type()` 方法，应该使用 `isinstance()` 或者 `x.type()` 方法：

In [5]:
x = torch.DoubleTensor([1, 1, 1])
print(type(x))
print(x.type())
print(isinstance(x, torch.DoubleTensor))

<class 'torch.Tensor'>
torch.DoubleTensor
True


### Tensor 的属性

每一个 Tensor 对象都有三个属性：`torch.dtype`, `torch.device`, `torch.layout` 以及 `requires_grad`

#### torch.dtype

Pytorch 有八种不同的 Tensor 类型，如下表所示：

| Data type | dtype | Tensor types|
| ---------:|----:|-----------:|
| 32-bit floating point | `torch.float32` or `torch.float` | `torch.*.FloatTensor` |
| 64-bit floating point | `torch.float64` or `torch.double` | `torch.*.DoubleTensor` |
| 16-bit floating point | `torch.float16` or `torch.half` | `torch.*.HalfTensor` |
| 8-bit integer (unsigned) | `torch.uint8` | `torch.*.ByteTensor` |
| 8-bit integer (signed) | `torch.int8` | `torch.*.CharTensor` |
| 16-bit integer (signed) | `torch.int16` or `torch.short` | `torch.*.ShortTensor` |
| 32-bit integer (signed) | `torch.int32` or `torch.int` | `torch.*.IntTensor` |
| 64-bit integer (signed) | `torch.int64` or `torch.long` | `torch.*.LongTensor` |

#### torch.device

torch.device 属性决定了数据分配内存的位置，目前推荐以下代码来决定不确定是否使用 GPU 运算：

```python
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
input = data.to(device)
model = MyModule(...).to(device)
```

#### torch.layout

决定存储方式，目前只支持 `torch.strided` 和 `torch.sparse_coo` 两种方式。

### requires_grad

是否累积梯度，默认为 False，且该属性为`或`运算方式。

### 与 numpy 中数据类型的互相转换

* `torch.tensor()` 方法支持从 np.ndarray 中复制数据并且生成一个 Tensor 对象；
* `torch.from_numpy()` 方法仅可从 np.ndarray 中生成一个 Tensor 对象，且两者在内存上共享; （文档如此解释，但是实验并不是这样）
* `Tensor.numpy()` 方法可以将一个 Tensor 对象转变成 np.ndarray 变量，且两者在内存上共享。

> **注意**：一个在计算图中的 Tensor 对象转化为 np.ndarray 变量，可用 `Tensor.detach()` 方法避免浅拷贝，`detach()` 方法会产生一个脱离于计算图的新 Tensor 对象，并且其 requires_grad=False

In [6]:
# convert data type between numpy and torch.Tensor
a = np.array([1., 1., 1.])
b = torch.tensor(a)
c = torch.from_numpy(a)
print(f'At first, a is {a}')
print(f'At first, b is {b}')
print(f'At first, c is {c}')
a[0] = -1
print(f'Now, a is {a}')
print(f'And b is {b}')
print(f'And c is {c}')

At first, a is [1. 1. 1.]
At first, b is tensor([ 1.,  1.,  1.], dtype=torch.float64)
At first, c is tensor([ 1.,  1.,  1.], dtype=torch.float64)
Now, a is [-1.  1.  1.]
And b is tensor([ 1.,  1.,  1.], dtype=torch.float64)
And c is tensor([-1.,  1.,  1.], dtype=torch.float64)


In [7]:
# convert data type between numpy and torch.Tensor
a = torch.rand(5, 4)
b = a.numpy()
print(b)

[[0.96112955 0.03798503 0.4426818  0.44538188]
 [0.61590683 0.75431883 0.9350943  0.6664036 ]
 [0.23648971 0.64038146 0.5608739  0.00237489]
 [0.5874882  0.44959515 0.4957475  0.98695034]
 [0.955158   0.91224027 0.02251083 0.92844963]]


#### 基本运算与 numpy 类似

In [8]:
x = torch.rand(5, 4)
y = torch.rand(5, 4)
c = 3

In [9]:
print(c * x)

tensor([[ 1.7482,  1.1246,  1.4485,  1.7168],
        [ 1.0103,  2.7181,  1.5097,  0.6454],
        [ 1.4979,  1.0452,  1.4803,  2.7384],
        [ 2.7554,  1.6346,  0.6852,  1.0243],
        [ 0.9178,  0.0983,  0.8559,  2.3039]])


In [10]:
print(x * y)

tensor([[ 0.0866,  0.1379,  0.3320,  0.1083],
        [ 0.0821,  0.3130,  0.0868,  0.0392],
        [ 0.0685,  0.2789,  0.1787,  0.0566],
        [ 0.7293,  0.2979,  0.1071,  0.0980],
        [ 0.2436,  0.0306,  0.2122,  0.3030]])


In [11]:
print(x + y)

tensor([[ 0.7313,  0.7428,  1.1705,  0.7615],
        [ 0.5807,  1.2514,  0.6758,  0.3974],
        [ 0.6366,  1.1490,  0.8557,  0.9748],
        [ 1.7126,  1.0916,  0.6974,  0.6285],
        [ 1.1023,  0.9661,  1.0291,  1.1626]])


In [12]:
print(x.add(y))

tensor([[ 0.7313,  0.7428,  1.1705,  0.7615],
        [ 0.5807,  1.2514,  0.6758,  0.3974],
        [ 0.6366,  1.1490,  0.8557,  0.9748],
        [ 1.7126,  1.0916,  0.6974,  0.6285],
        [ 1.1023,  0.9661,  1.0291,  1.1626]])


x.add_() 函数可以直接改变 x 的值

In [13]:
x.add_(y)

tensor([[ 0.7313,  0.7428,  1.1705,  0.7615],
        [ 0.5807,  1.2514,  0.6758,  0.3974],
        [ 0.6366,  1.1490,  0.8557,  0.9748],
        [ 1.7126,  1.0916,  0.6974,  0.6285],
        [ 1.1023,  0.9661,  1.0291,  1.1626]])

In [14]:
print(x)

tensor([[ 0.7313,  0.7428,  1.1705,  0.7615],
        [ 0.5807,  1.2514,  0.6758,  0.3974],
        [ 0.6366,  1.1490,  0.8557,  0.9748],
        [ 1.7126,  1.0916,  0.6974,  0.6285],
        [ 1.1023,  0.9661,  1.0291,  1.1626]])


#### 将 torch.Tensor 放到 GPU 上（该方式已可以弃用）

In [15]:
# checkout whether your machine supports GPU calculation
torch.cuda.is_available()

True

In [16]:
a = torch.rand(5, 4)
if torch.cuda.is_available():
    a = a.cuda()
print(a)

tensor([[ 0.6496,  0.7672,  0.0563,  0.2528],
        [ 0.5232,  0.6366,  0.0891,  0.7203],
        [ 0.4789,  0.0279,  0.3992,  0.5644],
        [ 0.8771,  0.7276,  0.3273,  0.5362],
        [ 0.2573,  0.5473,  0.1579,  0.2247]], device='cuda:0')


其余可用 cuda 功能的 API

In [17]:
torch.cuda.device_count()

2

In [18]:
b = torch.rand(2, 4)
if torch.cuda.is_available():
    with torch.cuda.device(1):
        print(f'Current GPU device: {torch.cuda.current_device()}')
        b = b.cuda()
print(b)

Current GPU device: 1
tensor([[ 0.3854,  0.7269,  0.7160,  0.5512],
        [ 0.6965,  0.1586,  0.1685,  0.1879]], device='cuda:1')


### 使用 nn.DataParallel 替代 multiprocessing

大多数涉及批量输入和多个GPU的情况应默认使用`DataParallel`来使用多个GPU。尽管有GIL的存在，单个python进程也可能使多个GPU饱和。

从0.1.9版本开始，大量的GPU(8+)可能未被充分利用。然而，这是一个已知的问题，也正在积极开发。和往常一样，测试你的用例吧。

调用`multiprocessing`来利用CUDA模型存在重要的注意事项；使用具有多处理功能的CUDA模型有重要的注意事项; 除非就是需要谨慎地满足数据处理需求，否则您的程序很可能会出现错误或未定义的行为。

### torch 的自动求导功能

torch 和大部分框架一样有着自动求导功能，

> 在0.4.0版本之前，只可以用 torch.autograd.Variable 创建可求导变量，
> 旧版本的 Variable 和 Tensor 本质上也没有什么区别，不过 Variable 会放在一个计算图里面，可以进行前向传播和反向传播以及求导
> 而在 0.4.0 版本之后，Tensor 和 Variable 合并为一个类

旧版本 Variable 示意图：![1.png](http://pytorch.org/tutorials/_images/Variable.png)

我们可以通过调用 Variable 对象的 **.data** 属性计算得到原始的 Tensor 变量（现在合并后已不推荐使用），而变量的累积梯度可以通过 **.grad** 属性获得。

自动求导中有一个重要的功能 function，即每个变量的 **.grad_fn** 属性，该属性记录了创造该变量的函数过程，因此用户起始创造的变量该属性为空。以下即为从 Tensor 中创造变量的示例，其中 requires_grad 表示是否计算梯度，默认值为 False

> 旧版本中除 requires_grad，还有 volatite 决定是否求导，现已经抛弃后一参数

In [19]:
x = torch.tensor([3.], requires_grad=True)
y = torch.tensor([5.], requires_grad=True)
print(f'Tensor x\'s .grad_fn is None: {x.grad_fn}')
z = 2 * x + y + 4
print(f'Tensor z\'s has .grad_fn: {z.grad_fn}')

Tensor x's .grad_fn is None: None
Tensor z's has .grad_fn: <AddBackward0 object at 0x7f956131ae80>


计算生成 z 的累积梯度值

In [20]:
z.backward()

最终打印关于各变量的梯度值

In [21]:
print(f'dz/dx: {x.grad}')
print(f'dz/dy: {y.grad}')

dz/dx: tensor([ 2.])
dz/dy: tensor([ 1.])


### 神经网络部分

所依赖的主要是 torch.nn 和 torch.nn.functional

torch.nn 里面有着所有的神经网络的层的操作，其用来构建网络，只有执行一次网络的运算才执行一次；

torch.nn.functional 里面包含的接口函数更加复杂，可以实现更灵活的操作。

In [22]:
from torch import nn
import torch.nn.functional as F

构建网络的类框架，以下为包含一个卷积层的神经网络。

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # add a convolutional layer
        self.conv1 = nn.Conv2d(1, 2, 3)
        self.conv2 = nn.Conv2d(2, 1, 4)
        # more information of various layers could be found in pytorch's manual
        
    def forward(self, x):
        # define forward propagation
        x = self.conv1(x)
        out = self.conv2(x)
        return out

实例化的网络对象可以直接打印其结构：

In [24]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 2, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(2, 1, kernel_size=(4, 4), stride=(1, 1))
)


In [25]:
len(list(net.parameters()))

4

In [26]:
input = torch.randn(1, 1, 10, 10)
out = net(input)
print(out)

tensor([[[[ 0.0928, -0.3174, -0.3287, -0.3080, -0.0933],
          [-0.3482,  0.0390, -0.0953, -0.1080, -0.1015],
          [-0.1584, -0.1685, -0.5402,  0.0240, -0.5067],
          [-0.2684,  0.1960,  0.0430,  0.0274, -0.3709],
          [-0.0219, -0.3011, -0.1487, -0.2501, -0.1467]]]])
